In [22]:
import pandas as pd
import numpy as np

import bz2
from datetime import datetime
import json
from urllib.request import urlopen

In [21]:
qardata.getQar()

NameError: name 'urlretrieve' is not defined

In [54]:
# Get denormalized-flights data
def get_denorm_data(from_param, to_param) :
    denorm_url = 'http://sfuelepcn1.thaiairways.co.th:3001/denormalized-flights?'
    skip = 'skip=0'
    limit = 'limit=0'
    parameter = denorm_url + from_param +'&' + to_param + '&' + skip + '&' + limit
    return pd.read_json(parameter)

# User input eOFP (Set Limit = 1000, default = 50)
def get_oefp_user_input(skip = '0', limit = '500') :
    skip = str(skip)
    limit = str(limit)

    url = "https://tgeofp.rtsp.us/api/v1/userinputs?skip="+skip+"&limit="+limit
    return pd.read_json(url)

# Get all OFP data (Set limit = 1000, default = 150)
def get_ofp() :
    url = "https://tgeofp.rtsp.us/api/v1/ofp?limit=1000"
    return pd.read_json(url)

# Get JSON of ofp data by specific flightplan id
def get_ofp_by_flightplan(flightplan) :
    try:
        url = "https://tgeofp.rtsp.us/api/v1/ofp/" + flightplan
        response = urlopen(url)
        data = json.loads(response.read())
        return data
    except Exception as e:
        return None

def create_fuelreport_df(denorm_df):
    fuel_report_df = denorm_df[denorm_df.fuelreport.notna()]['fuelreport'].apply(pd.Series)
    fuel_report_df.drop_duplicates(subset=['dep', 'flight_number', 'flight_date', 'aircraft_registration'], inplace=True)
    return fuel_report_df

def count_fuel_data(fuel_report_df):
    """Return DataFram contain number of Fuel report data each month"""
    
    fuel_summary_df = fuel_report_df.sort_values(['flight_date', 'flight_number']).reset_index()[['flight_date','flight_number','aircraft_registration','dep']]
    month = []
    year = []
    for date in fuel_summary_df.flight_date:
        date_time = pd.to_datetime(date)
        month.append(date_time.month)
        year.append(date_time.year)

    fuel_summary_df['month'] = month
    fuel_summary_df['year'] = year
    return fuel_summary_df.groupby(['year','month'])['flight_number'].count()

def merge_flightPlan_eofp(eofp) :
    """ger eofp dataFrame then use each flight plan to get information from flighplan database"""
    ## Initialize dataframe with eOFP
    df = eofp

    ## Get OFP data to create joint column on denorm
    for index, row in df.iterrows() :
        flightplan = df.iloc[index]["flightPlan"]
        
        # Get ofp flight information
        ofp_json = get_ofp_by_flightplan(flightplan)

        # Drop Unmatch FlightPlan ID
        if ofp_json == None : 
            df.drop(index = index, axis = 0)
            continue

        dep = ofp_json["flight_key"]['departure_aerodrome']['value']
        arr = ofp_json["flight_key"]['arrival_aerodrome']['value']
        flt_no = "THA" + ofp_json["flight_key"]["flight_number"]
        flt_date = datetime.strptime(ofp_json["flight_key"]["flight_date"],"%Y-%m-%dZ")
        flt_date = flt_date.strftime("%Y-%m-%dT%H:%M:%S.000Z")
        ac_reg = ofp_json["aircraft"]["aircraft_registration"]
        aircraft_registration = ac_reg[:2] + "-" + ac_reg[2:]
        imported_time = datetime.strptime(ofp_json["imported_time"], "%Y-%m-%dT%H:%M:%S.%fZ")
        #imported_time = imported_time.strftime("%Y-%m-%dT%H:%M:%S.000Z") ## TO String

        # Insert new column
        df.loc[index, "departure_aerodrome_icao_code"] = dep
        df.loc[index, "arrival_aerodrome_icao_code"] = arr
        df.loc[index, "flight_number"] = flt_no
        df.loc[index, "flight_date"] = flt_date
        df.loc[index, "aircraft_registration"] = aircraft_registration
        df.loc[index, "ofp_imported_time"] = imported_time
    
    ## Trim only eOFP data with new inserted column
    data_list = [
        "flightPlan","userInput","plannedCheckPoint","createdAt","updatedAt","ofp_imported_time",
        "departure_aerodrome_icao_code", "arrival_aerodrome_icao_code",
        "aircraft_registration", "flight_date", "flight_number",
        ]
    df = df[data_list]
    return df

In [73]:
# Adjust to/from date
from_param = 'from=2022-04-01T00:00:00.000Z'
to_param = 'to=2022-06-06T23:59:59.000Z'

# Create DataFrame

# Fuel Report
denorm_df = get_denorm_data(from_param, to_param)
fuel_report_df = create_fuelreport_df(denorm_df)

# OP data
op_data_df = pd.read_excel('./data/OpMovement.xlsx',sheet_name = 'OpMovement', usecols = 'A:H')
op_data_df['Date'] = op_data_df.Date.apply(pd.to_datetime)
years = []
months = []
for date in op_data_df['Date']:
    years.append(date.year)
    months.append(date.month)

op_data_df['year'] = years
op_data_df['month'] = months
op_data_df['Registration'] = 'HS-' + op_data_df.Registration

total_flights = op_data_df.groupby(['year', 'month'])['Flt No'].count()

# eOFP data

# initialize number or eOFP report to be downloaded
total_data = len(fuel_report_df)
total_loop = total_data // 500

# load first chunk of eOFP data
eofp_df = get_oefp_user_input()

for i in np.arange(total_loop):
    limit = 500
    skip = 500 + 500 * i
    chunk = get_oefp_user_input(skip=skip, limit=limit)
    
    # Don't append last chunk
    if len(chunk) < 500:
        break

    eofp_df = pd.concat([eofp_df, chunk], ignore_index= True)

eofp_df_merged = merge_flightPlan_eofp(eofp_df)

# TGFR data

tgfr_df = pd.read_csv('./data/Survey.csv',sep=';')
date_time = []
for date in tgfr_df['flightDate']:
    date_time.append(pd.to_datetime(date))
tgfr_df['flightDate'] = date_time
tgfr_df[['departure', 'arrival']] = tgfr_df['flightSector'].str.split('-', expand=True)
tgfr_columns = ['flightDate', 'flightNumber', 'acRegistration', 'departure', 'arrival',
    'plannedZFW', 'updatedZFW', 'ZFW',
    'plannedFlightTime', 
    'plannedTaxiFuel', 'plannedRampFuel', 'plannedTripFuel',
    'rampFuel','landingFuel','parkingFuel', 
    'offBlock', 'airborne', 'onGround', 'onBlock', 
    'takeOffFuel', 'planningType',
    'pfId', 'pmId', 'apuForParking']
tgfr_df = tgfr_df[tgfr_columns]

# QAR data

qar_df = pd.read_json('.\qar\qar.json')
sel_cols = ['aircraft_registration',
            'qar_start_time',
            'aircraft_type',
            'departure_aerodrome',
            'one_engine_taxi_out',
            'takeoff_flaps',
            'pack_off_takeoff',
            'acceleration_height',
            'climb_sd',
            'descent_sd',
            'landing_config_height',
            'reduced_flaps_landing',
            'idle_reverse_landing_performed',
            'landing_flaps',
            'one_engine_taxi_in',
            'apu_used_during_taxi_in']
qar_df = qar_df[sel_cols]

# convert qar start date to date column
qar_df['date'] = qar_df.qar_start_time.apply(pd.Series)
qar_df['date'] = qar_df.date.apply(pd.to_datetime)
qar_df.drop(columns='qar_start_time', inplace=True)

# set date column as index
qar_df.set_index(qar_df.date, drop=True, inplace=True)

qar_df.sort_index(ascending=True, inplace=True)
# select data for year 2021 and after
qar_df_2021 = qar_df.loc['2021-01-01':]
# Change aircraft type between B788 and B789
import qardata

for i, row in qar_df_2021.iterrows():
    qar_df_2021.loc[i,'aircraft_type'] = qardata.b787_type(row[0], row[1])

ValueError: Expected object or value

In [71]:
# fuel initiatives report